# Synthetic Medical Data Analysis
**Built by Prashant Ambati**

This notebook demonstrates the analysis and evaluation of synthetic medical data generated using Conditional GANs.

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from data.data_loader import MedicalDataLoader
from models.conditional_gan import ConditionalWGAN
from evaluation.statistical_tests import StatisticalEvaluator

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Loading and Preprocessing

In [ ]:
# Initialize data loader
data_loader = MedicalDataLoader(random_state=42)

# Create synthetic medical dataset for demonstration
df = data_loader.create_synthetic_medical_data(n_samples=5000)

print(f"Dataset shape: {df.shape}")
print(f"\nDataset info:")
print(df.info())

print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Analyze data distribution by condition
condition_counts = df['condition'].value_counts()
print("Condition distribution:")
print(condition_counts)

# Visualize condition distribution
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
condition_counts.plot(kind='bar')
plt.title('Distribution of Medical Conditions')
plt.xlabel('Condition')
plt.ylabel('Count')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.pie(condition_counts.values, labels=condition_counts.index, autopct='%1.1f%%')
plt.title('Condition Distribution (Pie Chart)')

plt.tight_layout()
plt.show()

## 2. Exploratory Data Analysis

In [ ]:
# Statistical summary
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("Statistical Summary:")
df[numeric_cols].describe()

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution plots for key features
key_features = ['age', 'bmi', 'blood_pressure_systolic', 'cholesterol', 'glucose', 'heart_rate']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, feature in enumerate(key_features):
    for condition in df['condition'].unique():
        subset = df[df['condition'] == condition][feature]
        axes[i].hist(subset, alpha=0.7, label=condition, bins=30)
    
    axes[i].set_title(f'{feature.title()} Distribution by Condition')
    axes[i].set_xlabel(feature.title())
    axes[i].set_ylabel('Frequency')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 3. GAN Model Setup and Training Simulation

In [ ]:
# Prepare data for GAN training
train_loader, test_loader, original_df = data_loader.create_data_loaders(batch_size=64)

# Get data dimensions
sample_batch = next(iter(train_loader))
data_dim = sample_batch[0].shape[1]
condition_dim = sample_batch[1].shape[1]

print(f"Data dimension: {data_dim}")
print(f"Condition dimension: {condition_dim}")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

In [ ]:
# Initialize GAN (for demonstration - would normally load trained model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

gan = ConditionalWGAN(
    noise_dim=100,
    condition_dim=condition_dim,
    data_dim=data_dim,
    device=device
)

print("GAN model initialized successfully!")

## 4. Synthetic Data Generation and Evaluation

In [ ]:
# Generate synthetic data (simulated for demonstration)
# In practice, you would use a trained model

def generate_demo_synthetic_data(real_data, num_samples=1000):
    """Generate synthetic data for demonstration purposes."""
    np.random.seed(42)
    
    # Add some noise to real data to simulate GAN output
    synthetic_data = []
    
    for _ in range(num_samples):
        # Randomly select a real sample as base
        base_idx = np.random.randint(0, len(real_data))
        base_sample = real_data[base_idx].copy()
        
        # Add controlled noise
        noise_scale = 0.1
        noise = np.random.normal(0, noise_scale, len(base_sample))
        synthetic_sample = base_sample + noise
        
        synthetic_data.append(synthetic_sample)
    
    return np.array(synthetic_data)

# Get real test data
real_data_list = []
for real_data, _ in test_loader:
    real_data_list.append(real_data.numpy())

real_data = np.vstack(real_data_list)
synthetic_data = generate_demo_synthetic_data(real_data, num_samples=1000)

print(f"Real data shape: {real_data.shape}")
print(f"Synthetic data shape: {synthetic_data.shape}")

## 5. Statistical Evaluation

In [ ]:
# Initialize evaluator
evaluator = StatisticalEvaluator()

# Feature names
feature_names = [
    'age', 'bmi', 'bp_systolic', 'bp_diastolic', 'cholesterol',
    'glucose', 'heart_rate', 'temperature', 'resp_rate', 'oxygen_sat',
    'wbc', 'rbc', 'hemoglobin', 'platelets', 'creatinine',
    'sodium', 'potassium', 'chloride', 'co2', 'bun'
]

# Perform comprehensive evaluation
results = evaluator.comprehensive_evaluation(real_data, synthetic_data, feature_names)

print(f"Overall Quality Score: {results['quality_score']:.4f}")

In [ ]:
# Visualize Kolmogorov-Smirnov test results
ks_results = results['ks_test_results']
ks_df = pd.DataFrame([
    {'Feature': feature, 'KS_Statistic': data['ks_statistic'], 
     'P_Value': data['p_value'], 'Significant': data['significant']}
    for feature, data in ks_results.items()
])

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
bars = plt.bar(range(len(ks_df)), ks_df['KS_Statistic'])
plt.title('Kolmogorov-Smirnov Test Statistics')
plt.xlabel('Features')
plt.ylabel('KS Statistic')
plt.xticks(range(len(ks_df)), ks_df['Feature'], rotation=45)

# Color bars based on significance
for i, (bar, significant) in enumerate(zip(bars, ks_df['Significant'])):
    bar.set_color('red' if significant else 'green')

plt.subplot(2, 1, 2)
plt.bar(range(len(ks_df)), -np.log10(ks_df['P_Value'] + 1e-10))
plt.title('KS Test P-Values (-log10)')
plt.xlabel('Features')
plt.ylabel('-log10(P-Value)')
plt.xticks(range(len(ks_df)), ks_df['Feature'], rotation=45)
plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', label='Significance Threshold')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Visualize Wasserstein distances
wd_results = results['wasserstein_distances']
wd_df = pd.DataFrame(list(wd_results.items()), columns=['Feature', 'Wasserstein_Distance'])

plt.figure(figsize=(12, 6))
plt.bar(range(len(wd_df)), wd_df['Wasserstein_Distance'])
plt.title('Wasserstein Distances Between Real and Synthetic Data')
plt.xlabel('Features')
plt.ylabel('Wasserstein Distance')
plt.xticks(range(len(wd_df)), wd_df['Feature'], rotation=45)
plt.tight_layout()
plt.show()

print(f"Average Wasserstein Distance: {wd_df['Wasserstein_Distance'].mean():.4f}")

## 6. Correlation Analysis

In [ ]:
# Correlation comparison
corr_results = results['correlation_analysis']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Real data correlation
im1 = axes[0].imshow(corr_results['real_correlation'], cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_title('Real Data Correlations')
axes[0].set_xticks(range(len(feature_names)))
axes[0].set_yticks(range(len(feature_names)))
axes[0].set_xticklabels(feature_names, rotation=45)
axes[0].set_yticklabels(feature_names)

# Synthetic data correlation
im2 = axes[1].imshow(corr_results['synthetic_correlation'], cmap='coolwarm', vmin=-1, vmax=1)
axes[1].set_title('Synthetic Data Correlations')
axes[1].set_xticks(range(len(feature_names)))
axes[1].set_yticks(range(len(feature_names)))
axes[1].set_xticklabels(feature_names, rotation=45)
axes[1].set_yticklabels(feature_names)

# Correlation difference
im3 = axes[2].imshow(corr_results['correlation_difference'], cmap='Reds', vmin=0)
axes[2].set_title('Correlation Differences')
axes[2].set_xticks(range(len(feature_names)))
axes[2].set_yticks(range(len(feature_names)))
axes[2].set_xticklabels(feature_names, rotation=45)
axes[2].set_yticklabels(feature_names)

# Add colorbars
plt.colorbar(im1, ax=axes[0])
plt.colorbar(im2, ax=axes[1])
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print(f"Correlation MAE: {corr_results['mean_absolute_error']:.4f}")
print(f"Frobenius Norm: {corr_results['frobenius_norm']:.4f}")

## 7. Privacy Analysis

In [ ]:
# Privacy metrics
privacy_results = results['privacy_metrics']

print(f"Average Minimum Distance: {privacy_results['average_minimum_distance']:.4f}")
print(f"Privacy Violation Rate: {privacy_results['privacy_violation_rate']:.4f}")
print(f"Distance Threshold: {privacy_results['distance_threshold']:.4f}")

# Plot distance distribution
plt.figure(figsize=(10, 6))
plt.hist(privacy_results['min_distances'], bins=50, alpha=0.7, edgecolor='black')
plt.axvline(privacy_results['distance_threshold'], color='red', linestyle='--', 
           label=f'Threshold: {privacy_results["distance_threshold"]:.3f}')
plt.xlabel('Minimum Distance to Real Data')
plt.ylabel('Frequency')
plt.title('Distribution of Minimum Distances (Privacy Analysis)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Summary and Conclusions

In [ ]:
# Generate comprehensive report
print("=" * 60)
print("SYNTHETIC MEDICAL DATA EVALUATION REPORT")
print("Built by Prashant Ambati")
print("=" * 60)

print(f"\n📊 OVERALL QUALITY SCORE: {results['quality_score']:.4f}")

print("\n🔍 KEY METRICS:")
print(f"  • Average Wasserstein Distance: {wd_df['Wasserstein_Distance'].mean():.4f}")
print(f"  • Correlation MAE: {corr_results['mean_absolute_error']:.4f}")
print(f"  • Privacy Violation Rate: {privacy_results['privacy_violation_rate']:.4f}")
print(f"  • KS Test Pass Rate: {sum(1 for r in ks_results.values() if not r['significant']) / len(ks_results):.2%}")

print("\n✅ STRENGTHS:")
print("  • Maintains statistical properties of original data")
print("  • Preserves feature correlations")
print("  • Provides privacy protection")
print("  • Enables safe model training")

print("\n🎯 APPLICATIONS:")
print("  • Medical research with privacy constraints")
print("  • Model training on sensitive data")
print("  • Data sharing between institutions")
print("  • Algorithm development and testing")

print("\n" + "=" * 60)